In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [8]:
p, i = np.meshgrid(np.arange(500), np.arange(512 // 2))
np.sin(p / 10000 ** (2 * i / 128)).shape

(256, 500)

In [9]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1:
            max_dims += 1
        #x좌표, y좌표 = np.meshgrid(x축 값들, y축 값들)
        #격자를 그리기 위한 X, Y 값들을 반환
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000 ** (2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000 ** (2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))

    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]


In [ ]:
embed_size = 512
max_steps = 500
vocab_size = 10000

embeddings = keras.layers.Embedding(vocab_size, embed_size)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)

encoder_input = keras.layer.Input(shape=[None], dtype=np.int32)
encoder_embeddings = embeddings(encoder_input)
encoder_in = positional_encoding(encoder_embeddings)


decoder_input = keras.layer.Input(shap=[None], dtype=np.int32)
decoder_embeddings = embeddings(decoder_input)
decoder_in = positional_encoding(decoder_embeddings)